# Looking at https://github.com/ytsvetko/metaphor

In [17]:
##all the imports...
#%pylab inline
import re
import math
import string
import nltk
import pprint
import matplotlib
import pandas as pd
import numpy as np
from nltk import word_tokenize
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Start by loading the .csv files and setting the columns.
### For these .csv files, downloaded from https://github.com/ytsvetko/metaphor inputs and saved as .csv with the first row added as "sample"

In [5]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet.head()

,sample
0,angry welt
1,bald assertion
2,bare outline
3,black humor
4,blind alley


In [6]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet['metaphor'] = 1
df_anmet['an_met'] = 1
df_anmet['an_nonmet'] = 0
df_anmet['svo_met'] = 0
df_anmet['svo_nonmet'] = 0
df_anmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,angry welt,1,1,0,0,0
1,bald assertion,1,1,0,0,0
2,bare outline,1,1,0,0,0
3,black humor,1,1,0,0,0
4,blind alley,1,1,0,0,0


In [7]:
df_annonmet = pd.read_csv("an_nonmets.csv", low_memory=False)
df_annonmet['metaphor'] = 0
df_annonmet['an_met'] = 0
df_annonmet['an_nonmet'] = 1
df_annonmet['svo_met'] = 0
df_annonmet['svo_nonmet'] = 0
df_annonmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,angry protester,0,0,1,0,0
1,bald eagle,0,0,1,0,0
2,big city,0,0,1,0,0
3,blind man,0,0,1,0,0
4,bloody nose,0,0,1,0,0


In [8]:
df_svomet = pd.read_csv("svo_mets.csv", low_memory=False)
df_svomet['metaphor'] = 1
df_svomet['an_met'] = 0
df_svomet['an_nonmet'] = 0
df_svomet['svo_met'] = 1
df_svomet['svo_nonmet'] = 0
df_svomet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,conversation turn subject,1,0,0,1,0
1,resumption bring relief,1,0,0,1,0
2,economy move direction,1,0,0,1,0
3,service meet expectation,1,0,0,1,0
4,material live dream,1,0,0,1,0


In [9]:
df_svononmet = pd.read_csv("svo_nonmets.csv", low_memory=False)
df_svononmet['metaphor'] = 0
df_svononmet['an_met'] = 0
df_svononmet['an_nonmet'] = 0
df_svononmet['svo_met'] = 0
df_svononmet['svo_nonmet'] = 1
df_svononmet.head()

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
0,car break *none,0,0,0,0,1
1,crowd scream *none,0,0,0,0,1
2,*person turn *none,0,0,0,0,1
3,foot slip *none,0,0,0,0,1
4,man shake head,0,0,0,0,1


In [10]:
#combine into one df
frames = [df_anmet, df_annonmet, df_svomet, df_svononmet]
df_combo = pd.concat(frames)

## Break down into training, dev, and test sets

In [14]:
random_index = np.random.permutation(df_combo.index)
random_index[:10]
df_combo.ix[random_index, ['sample', 'metaphor', 'an_met', 'an_nonmet', 'svo_met', 'svo_nonmet']]
df_shuffled = df_combo.ix[random_index, ['sample', 'metaphor', 'an_met', 'an_nonmet', 'svo_met', 'svo_nonmet']]
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled[500:510]

,sample,metaphor,an_met,an_nonmet,svo_met,svo_nonmet
500,fresh thinking,1,1,0,0,0
501,foggy night,0,0,1,0,0
502,advertiser pull ad,1,0,0,1,0
503,*pronoun grab towel,0,0,0,0,1
504,dead giveaway,1,1,0,0,0
505,dead animal,0,0,1,0,0
506,temperature break *number,1,0,0,1,0
507,*pronoun wear helmet,0,0,0,0,1
508,hairy problem,1,1,0,0,0
509,gold coin,0,0,1,0,0


In [15]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)
#train_size = round(rows*.6)
train_size = round(rows*.9)
#dev_size   = round(rows*.2)
dev_size   = round(rows*.1)
df_train = df_shuffled.loc[:train_size]
df_train.shape
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
df_dev.shape
df_test = df_shuffled.loc[dev_size+train_size:].reset_index(drop=True)
df_test.shape

Rows: 1644
Columns: 6


(0, 6)

## CountVectorizer
### Use count vecotrizer with df = 3 for unigram and bigrams to predict dev set

In [30]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', analyzer=u'word', min_df=3)
df_train = df_train.fillna("")
df_dev = df_dev.fillna("")
df_test = df_test.fillna("")

In [31]:
arr_train_feature_sparse = vec.fit_transform(df_train['sample'])
arr_train_feature_sparse
arr_train_feature = arr_train_feature_sparse.toarray()
feature_labels = vec.get_feature_names()

In [32]:
arr_dev_feature_sparse = vec.transform(df_dev["sample"])
arr_dev_feature = arr_dev_feature_sparse.toarray()

In [33]:
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor']) #defining features (from reviews) and passing in Category label
logreg_predictions = logreg_model.predict(arr_dev_feature)
accuracy_score(df_dev['metaphor'], logreg_predictions)

0.95731707317073167